In [1]:
import os
import sys
sys.path.append("..")
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import ipywidgets as widgets
from IPython.display import display
import numpy as np
from LowLevelModules.GeneralFunctions import get_work_dir, MyPlot
from LowLevelModules.GeneralFunctions import TRMData
from scipy.signal import convolve
from scipy.optimize import curve_fit

In [2]:
from LowLevelModules.ImportSDT import SdtFile

class TRMData(SdtFile):
    def __init__(self, path):
        super().__init__(path)
        self.time = self.times[0]
        self.info = self.measure_info[0]
        self.x = self.time*1e9
        self.y = self.data[0][0]
        self.bin_size = (self.time[-1]-self.time[0])/(len(self.time)-1)

        #trim data
        self.x = self.x[self.y>0]
        self.y = self.y[self.y>0]
        trim = -1 * int(len(self.y)*0.04)
        self.x = self.x[:trim]
        self.y = self.y[:trim]
        
        #shift x axis
        self.x -= self.x[0]

        #count rate
        info_ctr = self.info['StopInfo']
        min_sync_rate = info_ctr['min_sync_rate'][0]
        min_cfd_rate = info_ctr['min_cfd_rate'][0]
        min_tac_rate = info_ctr['min_tac_rate'][0]
        max_sync_rate = info_ctr['max_sync_rate'][0]
        max_cfd_rate = info_ctr['max_cfd_rate'][0]
        max_tac_rate = info_ctr['max_tac_rate'][0]
        self.col_t = info_ctr['stop_time'][0]

        self.APD1_rate = (min_cfd_rate + max_cfd_rate) / 2
        self.APD1_rate_alt = (min_tac_rate + max_tac_rate) / 2
        self.laser_rep_rate = (min_sync_rate + max_sync_rate) / 2
        self.APD2_rate = self.laser_rep_rate
        
        #normalization
        self.total_counts = np.sum(self.y)      

In [6]:
from matplotlib.pyplot import subplots

#other functions
def exp_fold(xe, t_shift, amp, gamma):
    return (1 + 1/(np.exp(gamma * t_laser) - 1)) * amp * np.exp(-gamma*(xe-t_shift))
    
def Decay_1_exp(xd, t0, a1, g1):
    fit_raw = exp_fold(xd, t0, a1, g1)  
    return fit_raw

def Decay_2_exp(xd, t0, a1, g1, a2, g2):
    # take into account folding of data in form of exponential series
    exp1 = exp_fold(xd, t0, a1, g1)
    exp2 = exp_fold(xd, t0, a2, g2)
    fit_raw = exp1 + exp2
    # convolve with IRF
    fit_conv = convolve(fit_raw, irf.y, 'same') / sum(irf.y)
    return fit_conv

def Decay_3_exp(xd, t0, a1, g1, a2, g2, a3, g3):
    # take into account folding of data in form of exponential series
    exp1 = exp_fold(xd, t0, a1, g1)
    exp2 = exp_fold(xd, t0, a2, g2)
    exp3 = exp_fold(xd, t0, a3, g3)
    fit_raw = exp1 + exp2 + exp3
    # convolve with IRF
    #fit_conv = signal.convolve(raw_fit, irf.y,'same') / sum(irf.y)
    return fit_raw

In [12]:
wdir = r"Z:\Projects\WSe2\2020\02 february\20200224 - WSe2 hBN gated\5K data\lifetime"

filenames = [f[:-4] for f in os.listdir(wdir) if '.sdt' in f]
file_choice = widgets.RadioButtons(options=filenames,
                                     value=filenames[0],
                                     description='File')
decay_choice = widgets.RadioButtons(options=[1, 2, 3],
                                     value=2,
                                     description='# Exp')
display(file_choice)
display(decay_choice)

RadioButtons(description='File', options=('WSe2hBN_TRM_spot12_140nW_728exc', 'WSe2hBN_TRM_spot12_140nW_728exc_…

RadioButtons(description='# Exp', index=1, options=(1, 2, 3), value=2)

In [39]:
import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

sdt_data_path = wdir + '//' + file_choice.value + '.sdt'
data = TRMData(sdt_data_path)

bckg_path = r'Z:\Projects\WSe2\2020\02 february\background_2020_02_12.sdt'
bckg = TRMData(bckg_path)

irf_path = r"Z:\Projects\WSe2\2020\02 february\IRF_mira_60kHz.sdt"
irf = TRMData(irf_path)

#remove background contribution
bckg_rate = np.mean(bckg.y) / bckg.col_t
data.y = data.y - bckg_rate * data.col_t

#rescale IRF data
irf.y = irf.y / np.sum(irf.y) # np.max(irf.y) * np.max(data.y)

#fitting
t_laser = 1e9 / irf.laser_rep_rate
t_start = 1.3
idx = data.x>t_start
data_trunc_x = data.x[idx]
data_trunc_y = data.y[idx]
irf_trunc = irf.y[irf.y>1e-7]

## choose fitting function
func_choice = {1: Decay_1_exp, 2: Decay_2_exp, 3: Decay_3_exp}
func = func_choice[decay_choice.value]

## start guesses
rate_guess = t_laser / (np.max(data.y) - np.min(data.y))
A_guess = np.max(data.y)

# define initial guess and bounds
p00 = [0, A_guess, rate_guess]
bds_low = [0, 0, 0.01]
bds_high = [2, 10*A_guess, 100]

if decay_choice.value > 1:
    p00.extend([0.2*A_guess, 0.4*rate_guess])
    bds_low.extend([0, 0.01])
    bds_high.extend([10*A_guess, 100])

if decay_choice.value > 2:
    p00.extend([0.2*A_guess, 0.1*rate_guess])
    bds_low.extend([0, 0.01])
    bds_high.extend([10*A_guess, 100])

# do the fit
coeffs, _ = curve_fit(func, data_trunc_x, data_trunc_y, p0=p00, bounds=(bds_low, bds_high))
y_fit = func(data_trunc_x, *coeffs)

pl = MyPlot(1, 1, xlabel='Time (ns)', ylabel='Counts (a.u.)')
pl.semilogy(data.x, data.y, 'bo', markersize=3, label='data')
#pl.semilogy(irf.x, irf.y, 'g', label='IRF')
pl.semilogy(data_trunc_x, y_fit, 'r', label='fit')
#pl.ax.set_ylim([0.95*np.min(data.y), 1.1*np.max(data.y)])
pl.ax.legend(prop={'size': 18}, frameon=False)
pl.ax.text(8, 0.4*np.max(data.y), f"γf = {np.round(coeffs[2],3)} ns-1", size='x-large')
if decay_choice.value > 1:
    pl.ax.text(8, 0.3*np.max(data.y), f"γs = {np.round(coeffs[4],3)} ns-1", size='x-large')
plt.tight_layout()
#pl.fig.savefig(wdir + '\\' + file_choice.value + '.png', format='png', bbox_inches='tight')

[0, 36628.23981284142, 0.000359708498439419] [0, 0, 0.01] [2, 366282.3981284142, 100]


ValueError: `x0` is infeasible.

In [35]:
coeffs

array([1.00000000e-10, 1.20234046e+03, 1.81131439e-02])

In [7]:
import plotly.express as px
import plotly.graph_objs as go

In [8]:
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected = True)

def go_layout():
    lay = go.Layout(title="Sine wave")
    return

def go_plot(xdata, ydata):
    data = go.Scatter(x=xdata, y=ydata)
    return {'data': [data]}

pl = go_plot(data.x, data.y)
iplot(pl)

In [9]:
import pandas as pd
df = pd.DataFrame({'Time (ns)': data.x, 'Counts (a.u.)': data.y, 'IRF': irf.y})
fig = px.scatter(df, x="Time (ns)", y="Counts (a.u.)", log_y=True,  template="plotly_dark")
iplot(fig)

ValueError: arrays must all be same length